# Спонсорство та підтримка Рожанчука Богдана
#https://github.com/BogdanJeN/Geo/blob/main/Lab_06/lab_06.ipynb

In [2]:
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
from shapely.geometry import Point, mapping
import json

# Load soil data
clay_path = '../clay.tif'
sand_path = '../sand.tif'
density_path = '../density.tif'

clay_raster = rasterio.open(clay_path)
sand_raster = rasterio.open(sand_path)
density_raster = rasterio.open(density_path)

# Load field centroids
fields_path = '../field_centroids.geojson'
fields = gpd.read_file(fields_path)

# Load moisture data
moisture_data = rasterio.open('../soil_moisture.tif')

# Create a list to store the field information
field_list = []

# Loop through each field and extract the relevant soil and moisture data
for index, row in fields.iterrows():
    # Get the field name and centroid coordinates
    field_id = row['id']
    field_name = row['Name']
    centroid = row['geometry'].centroid
    lat, lon = centroid.y, centroid.x

    # Get the pixel coordinates of the centroid
    row, col = clay_raster.index(lon, lat)

    # Extract soil data for the centroid
    clay = clay_raster.read(1)[row, col]
    sand = sand_raster.read(1)[row, col]
    density = density_raster.read(1)[row, col]

    # Get the pixel coordinates of the centroid for the moisture data
    row, col = sand_raster.index(lon, lat)

    # Extract moisture data
    moisture = moisture_data.read(1)[row, col]

    # Create a dictionary with the field information
    field_dict = {
        'type': 'Feature',
        'properties': {
            'id': field_id,
            'name': field_name,
            'soil': {
                'clay': float(clay),
                'sand': float(sand),
                'density': int(density)
            },
            'soil_moisture': int(moisture),
            'coordinates': {
                'lat': float(lat),
                'lng': float(lon)
            }
        },
        'geometry': mapping(centroid)
    }

    # Add the field dictionary to the list
    field_list.append(field_dict)

# Convert the field list to a GeoJSON FeatureCollection
feature_collection = {
    'type': 'FeatureCollection',
    'features': field_list
}

# Write the GeoJSON to a file
with open('field_data.geojson', 'w') as f:
    json.dump(feature_collection, f, indent=2)

{
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {
        "id": 1,
        "name": "Pin4",
        "soil": {
          "clay": 21.0,
          "sand": 38.0,
          "density": 1149
        },
        "soil_moisture": 2785,
        "coordinates": {
          "lat": 49.995266264307425,
          "lng": 30.184082372130998
        }
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          30.184082372130998,
          49.995266264307425
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "id": 2,
        "name": "Pin17",
        "soil": {
          "clay": 21.0,
          "sand": 37.0,
          "density": 1212
        },
        "soil_moisture": 2813,
        "coordinates": {
          "lat": 49.98931843919832,
          "lng": 30.250268297428843
        }
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          30.250268297428843,
          49.98931843919832
        ]
      }
    },
    {
      "type": "Feature",
      "properties": {
        "id": 3,
        "name": "Shev7-8-9",
        "soil": {
          "clay": 22.0,
          "sand": 35.0,
          "density": 1184
        },
        "soil_moisture": 3158,
        "coordinates": {
          "lat": 50.01055996618192,
          "lng": 30.333908293755627
        }
      },
      "geometry": {
        "type": "Point",
        "coordinates": [
          30.333908293755627,
          50.01055996618192
        ]
      }
    }